**The news sentiment scraper**

This mini-project creates a vibe check for a specific topic on a site like hacker news or reddit


In [ ]:
import os
import requests;
from openai import OpenAI 
from dotenv import load_dotenv 
from IPython.display import Markdown, display

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check if the API key is valid
if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-or-"):
    print("An API key was found, but it doesn't start sk-or-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

# 1. SETUP: Configure your LLM
client = OpenAI(base_url="https://openrouter.ai/api/v1" , api_key=api_key)

def get_hn_vibe(topic):
    print(f"--- Checking the vibe for: {topic} ---")
    
    # 2. FETCH: Use the Algolia API to get the top 5 stories
    # Documentation: https://hn.algolia.com/api
    url = f"https://hn.algolia.com/api/v1/search?query={topic}&tags=story&hitsPerPage=5"
    response = requests.get(url)
    data = response.json()
    
    # 3. EXTRACT: Pull out just the titles
    titles = [post['title'] for post in data['hits']]
    
    if not titles:
        return "No stories found for that topic."

    # Format titles into a single string for the prompt
    formatted_titles = "\n".join([f"- {t}" for t in titles])
    print(f"Extracted Titles:\n{formatted_titles}\n")

    # 4. LLM CALL: Aggregate and analyze
    stream = client.chat.completions.create(
        model="openai/gpt-4o-mini", 
        messages=[
            {
                "role": "system", 
                "content": """
                You are a trend analyst. look at these titles and tell me if the general sentiment 
                is optimistic, skeptical, or fearful. explain why in one sentence.
                """
            },
            {
                "role": "user", 
                "content": f"Here are the top 5 recent titles on Hacker News regarding '{topic}':\n\n{formatted_titles}"
            }
        ],
        stream=True
    )
    display_handle = display(Markdown(""), display_id=True)
    response = ""
    for chunk in stream:
        delta = chunk.choices[0].delta.content or ""
        response += delta
        display_handle.update(Markdown(response))


    

# 5. EXECUTION
if __name__ == "__main__":
    topic_to_check = "AI regulation"
    get_hn_vibe(topic_to_check)



    

    
    

API key found and looks good so far!
--- Checking the vibe for: AI regulation ---
Extracted Titles:
- Tech Titans Amass Multimillion-Dollar War Chests to Fight AI Regulation
- OpenAI Lobbied the E.U. To Water Down AI Regulation
- GOP sneaks decade-long AI regulation ban into spending bill
- AI regulations are crony capitalism
- Sam Altman Says OpenAI Will Leave the EU If There's Any Real AI Regulation



The general sentiment is skeptical. This is reflected in the recurring theme of powerful companies actively resisting and undermining regulations designed for AI, suggesting a belief that such regulations will impose significant constraints on innovation and profitability.